### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import copy
import random
from PIL import Image
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# Colabのローカルファイルに自作データをコピー（初めからローカルファイルにアップロードする場合は不要）
!cp -r "/content/drive/MyDrive/HandSigns/" "/content/HandSigns/"

In [5]:
# 自作データをリサイズして、ファイル構造を保ったまま新しいフォルダに保存する関数を定義
def resize_images(base_dir, new_base_dir, image_size=(32, 32)):
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        new_folder_path = os.path.join(new_base_dir, folder)
        # リサイズした自作データを保存する新しいフォルダを作成
        os.makedirs(new_folder_path, exist_ok=True)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            # 拡張子の確認
            if file_path.endswith(('.jpg', '.jpeg')):
                # 画像を展開
                img = Image.open(file_path)
                # 画像をリサイズ
                img_resized = img.resize(image_size)
                # リサイズした画像を新しいフォルダに保存
                img_resized.save(os.path.join(new_folder_path, filename))

# 学習用の自作データのディレクトリを設定
train_base_dir = '/content/HandSigns/train'
# テスト用の自作データのディレクトリを設定
test_base_dir = '/content/HandSigns/test'

# リサイズした学習用の自作データを保存するディレクトリを設定
new_train_base_dir = '/content/HandSigns/resized_train_32'
# リサイズしたテスト用の自作データを保存するディレクトリを設定
new_test_base_dir = '/content/HandSigns/resized_test_32'
# リサイズする大きさを設定
image_size = (32, 32)

# 学習用の自作データをリサイズ
resize_images(train_base_dir, new_train_base_dir, image_size)
# テスト用の自作データをリサイズ
resize_images(test_base_dir, new_test_base_dir, image_size)

In [6]:
# 平均値と標準偏差を計算するためのHandSignsの前処理を定義
pre_transform_handsigns = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのHandSignsデータセットの前処理を定義
pre_train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=pre_transform_handsigns)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_handsigns = 0.0
pre_std_handsigns = 0.0
pre_total_samples_handsigns = len(pre_train_dataset_handsigns)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_handsigns:
    pre_image, _ = data
    pre_mean_handsigns += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_handsigns += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_handsigns /= pre_total_samples_handsigns
pre_std_handsigns /= pre_total_samples_handsigns

print("データセット全体の平均値: ", pre_mean_handsigns)
print("データセット全体の標準偏差: ", pre_std_handsigns)

データセット全体の平均値:  tensor([0.6954, 0.6754, 0.6450])
データセット全体の標準偏差:  tensor([0.1331, 0.1565, 0.1689])


In [7]:
# 学習用のHandSignsデータセットの前処理を定義
train_transform_handsigns = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# テスト用のHandSignsデータセットの前処理を定義
test_transform_handsigns = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_handsigns, std=pre_std_handsigns)
    ])

# 学習用のHandSignsデータセットの読み込み
train_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_train_32", transform=train_transform_handsigns)
# テスト用のHandSignsデータセットの読み込み
test_dataset_handsigns = torchvision.datasets.ImageFolder("/content/HandSigns/resized_test_32", transform=test_transform_handsigns)

# 学習用のHandSignsデータローダーを作成
train_loader_handsigns = torch.utils.data.DataLoader(dataset=train_dataset_handsigns, batch_size=16, shuffle=True, num_workers=2)
# テスト用のHandSignsデータローダーを作成
test_loader_handsigns = torch.utils.data.DataLoader(dataset=test_dataset_handsigns, batch_size=16, shuffle=False, num_workers=2)

In [8]:
# 学習データのクラス名とラベルの対応を確認
print(train_dataset_handsigns.class_to_idx)
# テストデータのクラス名とラベルの対応を確認
print(test_dataset_handsigns.class_to_idx)

{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}
{'100000': 0, '100001': 1, '100111': 2, '101000': 3, '110000': 4, '110001': 5, '111000': 6, '111001': 7, '111100': 8, '111111': 9}


In [9]:
# イテレータを作成
data_iter = iter(train_loader_handsigns)

# データを1バッチ分取得
images, labels = next(data_iter)

# 取得したバッチの画像とラベルの情報を確認
print("画像の形状:", images.shape)  # (バッチサイズ, チャンネル数, 高さ, 幅)
print("ラベル:", labels)

画像の形状: torch.Size([16, 3, 32, 32])
ラベル: tensor([3, 7, 4, 1, 3, 5, 6, 9, 9, 5, 3, 0, 4, 0, 2, 1])


## モデルの実装

In [10]:
# 1×1のサブネットワーク獲得用の畳み込みを定義
def supermaskconv1x1(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3のサブネットワーク獲得用の畳み込みを定義
def supermaskconv3x3(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [11]:
# サブネットワーク獲得用のバッチ正則化として、非アフィン正規化を使用する（学習可能なパラメータを使用しない）
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

In [12]:
# 各重みにスコアを付与したスコアをソートしてtop k%を使用することでサブネットワークを獲得する
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # スコアを複製する
        out = scores.clone()
        # スコアを昇順でソートする
        _, idx = scores.flatten().sort()
        # top k%以下のスコアの数
        j = int((1 - k) * scores.numel())
        # flat_outとoutは同じメモリを参照する（flat_outを変更するとoutにも影響する）
        flat_out = out.flatten()
        # top k%の要素を1にする
        flat_out[idx[j:]] = 1
        # top k%以外の要素を0にする
        flat_out[idx[:j]] = 0
        return out

    @staticmethod
    def backward(ctx, g):
        # 逆伝播時に勾配gをそのまま伝える
        return g, None

In [13]:
# サブネットワーク獲得用の畳み込みを定義
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 重みと同じ形状のスコアを用意
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        # スコアを一様分布で初期化
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))
        # 重みの勾配を無効化する
        self.weight.requires_grad = False

    def _init_conv(self):
        # 重みを一様分布で初期化
        if self.init == 'kaiming_uniform':
            nn.init.kaiming_uniform_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを正規分布で初期化
        elif self.init == 'kaiming_normal':
            nn.init.kaiming_normal_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを符号つき定数で初期化
        elif self.init == 'signed_constant':
            fan = nn.init._calculate_correct_fan(self.weight, mode='fan_out')
            gain = nn.init.calculate_gain('relu')
            std = gain / math.sqrt(fan)
            self.weight.data = self.weight.data.sign() * std

    def set_init(self, init):
        # 初期化手法を設定
        self.init = init

    def set_prune_rate(self, prune_rate):
        # 刈り込み率を設定
        self.prune_rate = prune_rate

    @property
    def clamped_scores(self):
        # スコアとして非負の値を返すようにする（重要度を表す）
        return self.scores.abs()

    def forward(self, x):
        # サブネットワークを獲得
        subnet = GetSubnet.apply(self.clamped_scores, 1 - self.prune_rate)
        # サブネットワークでマスク
        w = self.weight * subnet
        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [14]:
# サブネットワーク獲得用のResidual Blocksを定義
class SupermaskBasicBlock(nn.Module):
    # 入力チャンネル数に対する出力チャンネル数の倍率
    expansion = 1

    def __init__(self, in_channels, channels, stride=1):
        super().__init__()
        self.conv1 = supermaskconv3x3(in_channels, channels, stride)
        self.bn1 = NonAffineBatchNorm(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = supermaskconv3x3(channels, channels)
        self.bn2 = NonAffineBatchNorm(channels)
        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != channels * self.expansion or stride > 1:
            self.shortcut = nn.Sequential(
                supermaskconv1x1(in_channels, channels * self.expansion, stride),
                NonAffineBatchNorm(channels * self.expansion)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        # 残差写像と恒等写像の和を計算
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [15]:
# サブネットワーク獲得用のResNetを定義
class SupermaskResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = SupermaskConv(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = NonAffineBatchNorm(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # Residual Blocks(in_channels=64)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        # Residual Blocks(in_channels=128)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        # Residual Blocks(in_channels=256)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # Residual Blocks(in_channels=512)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # 重みの初期化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 初期化手法を設定
                m.set_init(init)
                # 刈り込み率を設定
                m.set_prune_rate(prune_rate)
                # 重みを初期化
                m._init_conv()

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, channels, blocks, stride):
        layers = []
        # 最初のResidual Block（stride=stride）
        layers.append(block(self.in_channels, channels, stride))
        # 残りのResidual Block（stride=1）
        self.in_channels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [16]:
# モデルを呼び出す関数を定義
def supermaskresnet34_10():
    return SupermaskResNet(SupermaskBasicBlock, [3, 4, 6, 3], num_classes=10)

## 学習と評価（100 epochs, init=kaiming_uniform）

### prune_rate=0.999

In [17]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.999_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 9.4 %, Loss: 2.6561
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 11.5 %, Loss: 2.3041
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.5 %, Loss: 2.2920
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 11.3 %, Loss: 2.7923
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 10.7 %, Loss: 2.2827
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 9.9 %, Loss: 2.1400
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 9.8 %, Loss: 2.4461
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 9.4 %, Loss: 2.7943
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 9.1 %, Loss: 2.8085
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 9.8 %, Loss: 2.6040
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 10.2 %, Loss: 2.6091
Epoch [1

In [18]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [19]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 33.0 %


In [20]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [21]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.995_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 9.2 %, Loss: 2.3384
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 9.9 %, Loss: 2.0330
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 9.1 %, Loss: 2.7549
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 10.6 %, Loss: 2.5783
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 11.4 %, Loss: 2.4820
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 12.0 %, Loss: 2.3080
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 13.3 %, Loss: 2.1143
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 16.8 %, Loss: 2.0967
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 16.5 %, Loss: 2.1354
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 17.8 %, Loss: 1.7924
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 18.4 %, Loss: 2.3235
Epoch

In [22]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [23]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 39.0 %


In [24]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [25]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.99_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 11.4 %, Loss: 2.4544
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 10.7 %, Loss: 2.5997
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.3 %, Loss: 2.2572
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 12.2 %, Loss: 2.3447
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 13.0 %, Loss: 1.9188
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 14.8 %, Loss: 2.1363
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 18.8 %, Loss: 2.2622
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 15.3 %, Loss: 2.6448
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 16.6 %, Loss: 2.3792
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 16.1 %, Loss: 2.3436
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 20.4 %, Loss: 1.8948
Ep

In [26]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [27]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 65.0 %


In [28]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [29]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.95_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 10.8 %, Loss: 2.1833
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 13.0 %, Loss: 2.3871
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 14.1 %, Loss: 2.1690
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 17.9 %, Loss: 2.3076
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 18.7 %, Loss: 2.1393
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 23.7 %, Loss: 1.9862
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 20.8 %, Loss: 2.3272
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 19.7 %, Loss: 2.1447
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 26.2 %, Loss: 1.7567
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 26.0 %, Loss: 2.5507
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 24.9 %, Loss: 2.0901
Ep

In [30]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [31]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 94.0 %


In [32]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [33]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.9_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 10.7 %, Loss: 2.7426
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 11.4 %, Loss: 2.3304
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 12.5 %, Loss: 2.4186
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 15.8 %, Loss: 2.7029
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 18.2 %, Loss: 2.1450
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 20.9 %, Loss: 2.1912
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 21.5 %, Loss: 2.3502
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 24.5 %, Loss: 1.9980
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 26.0 %, Loss: 2.6359
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 25.3 %, Loss: 2.7303
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 25.9 %, Loss: 2.3038
Ep

In [34]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [35]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 94.0 %


In [36]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [37]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.7_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 12.3 %, Loss: 2.7130
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 11.3 %, Loss: 2.3772
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 14.1 %, Loss: 2.2438
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 15.2 %, Loss: 1.8471
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.1 %, Loss: 2.3982
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 23.4 %, Loss: 2.1828
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 27.4 %, Loss: 1.6109
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 28.5 %, Loss: 1.8458
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 30.6 %, Loss: 1.8025
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 32.7 %, Loss: 1.8644
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 35.2 %, Loss: 1.9019
Ep

In [38]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [39]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 100.0 %


In [40]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [41]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.5_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 11.4 %, Loss: 2.2509
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 14.8 %, Loss: 2.1453
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 13.0 %, Loss: 2.1699
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 14.4 %, Loss: 2.1379
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.0 %, Loss: 2.0506
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 20.9 %, Loss: 2.8977
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 25.5 %, Loss: 2.3165
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 29.4 %, Loss: 1.4169
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 30.2 %, Loss: 1.5391
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 31.5 %, Loss: 1.8601
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 31.7 %, Loss: 3.0214
Ep

In [42]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [43]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 100.0 %


In [44]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=kaiming_normal）

### prune_rate=0.999

In [45]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.999_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.9 %, Loss: 2.4244
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 9.2 %, Loss: 2.4332
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 9.5 %, Loss: 2.4279
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 11.4 %, Loss: 2.4653
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 9.1 %, Loss: 2.7796
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 10.5 %, Loss: 2.9622
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 11.1 %, Loss: 2.8458
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 9.0 %, Loss: 2.3695
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 10.5 %, Loss: 2.9206
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 10.3 %, Loss: 2.4210
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 10.3 %, Loss: 2.2055
Epoch [

In [46]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [47]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 33.0 %


In [48]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [49]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.995_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 12.6 %, Loss: 2.9658
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 9.9 %, Loss: 2.5287
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 9.5 %, Loss: 2.4263
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 9.4 %, Loss: 2.8260
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 10.3 %, Loss: 2.1261
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 9.9 %, Loss: 2.2711
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 11.2 %, Loss: 2.3237
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 10.1 %, Loss: 2.4472
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 15.0 %, Loss: 2.3553
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 12.6 %, Loss: 2.7261
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 15.8 %, Loss: 2.4239
Epoch 

In [50]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [51]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 19.0 %


In [52]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [53]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.99_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 11.4 %, Loss: 2.8201
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 10.9 %, Loss: 2.9234
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.8 %, Loss: 2.2081
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 10.2 %, Loss: 2.0378
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 15.2 %, Loss: 2.0522
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 16.7 %, Loss: 1.9727
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 17.0 %, Loss: 2.3416
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 20.0 %, Loss: 2.9359
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 19.1 %, Loss: 2.0394
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 20.4 %, Loss: 2.0582
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 20.0 %, Loss: 2.5745
Ep

In [54]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [55]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 65.0 %


In [56]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [57]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.95_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 12.0 %, Loss: 2.5930
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 11.3 %, Loss: 2.0776
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 13.2 %, Loss: 2.6155
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 13.3 %, Loss: 2.6703
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 17.7 %, Loss: 2.8983
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 18.8 %, Loss: 2.1373
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 21.5 %, Loss: 2.0118
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 23.0 %, Loss: 2.0986
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 24.2 %, Loss: 2.1600
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 27.9 %, Loss: 2.1144
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 26.9 %, Loss: 2.4410
Ep

In [58]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [59]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 95.0 %


In [60]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [61]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.9_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 9.5 %, Loss: 2.2906
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 10.9 %, Loss: 2.1757
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 11.9 %, Loss: 2.3368
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 14.4 %, Loss: 2.2715
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.4 %, Loss: 1.8737
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 20.2 %, Loss: 2.0909
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 23.2 %, Loss: 2.4710
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 24.1 %, Loss: 2.2710
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 26.5 %, Loss: 2.0768
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 25.4 %, Loss: 1.9768
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 21.8 %, Loss: 2.1086
Epo

In [62]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [63]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 75.0 %


In [64]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [65]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.7_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 11.4 %, Loss: 2.5409
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.3 %, Loss: 2.5470
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 14.7 %, Loss: 2.5714
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 16.9 %, Loss: 2.3049
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 21.5 %, Loss: 1.9998
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 20.8 %, Loss: 1.6787
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 25.6 %, Loss: 2.4197
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 28.4 %, Loss: 3.2634
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 26.9 %, Loss: 1.6349
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 29.4 %, Loss: 1.8110
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 33.2 %, Loss: 1.9630
Ep

In [66]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [67]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 97.0 %


In [68]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [69]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.5_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 10.9 %, Loss: 2.4328
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.4 %, Loss: 2.3602
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 12.7 %, Loss: 2.2993
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 16.6 %, Loss: 2.7244
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 18.8 %, Loss: 2.3630
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 23.4 %, Loss: 1.4782
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 25.0 %, Loss: 1.8045
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 29.3 %, Loss: 1.5126
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 32.6 %, Loss: 1.8221
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 32.5 %, Loss: 1.8354
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 33.2 %, Loss: 1.8602
Ep

In [70]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [71]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 95.0 %


In [72]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=signed_constant）

### prune_rate=0.999

In [73]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.999_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 10.7 %, Loss: 3.1337
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 10.1 %, Loss: 3.0670
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.2 %, Loss: 2.3785
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 10.8 %, Loss: 2.5831
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 11.2 %, Loss: 2.5722
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 10.6 %, Loss: 2.2076
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 10.6 %, Loss: 2.3607
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 11.8 %, Loss: 2.1583
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 10.8 %, Loss: 2.5664
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 11.2 %, Loss: 2.4911
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 10.8 %, Loss: 2.2807
Ep

In [74]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [75]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 12.0 %


In [76]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [77]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.995_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 9.2 %, Loss: 2.4906
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 10.3 %, Loss: 2.6844
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.3 %, Loss: 2.6650
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 9.8 %, Loss: 2.2536
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 12.3 %, Loss: 2.2387
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 12.7 %, Loss: 2.0721
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 16.4 %, Loss: 2.5295
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 14.0 %, Loss: 2.5245
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 13.7 %, Loss: 2.1690
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 15.7 %, Loss: 2.5509
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 17.8 %, Loss: 2.2300
Epoc

In [78]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [79]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 72.0 %


In [80]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [81]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.99_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 9.6 %, Loss: 2.1930
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 11.0 %, Loss: 2.3162
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.9 %, Loss: 2.7282
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 11.3 %, Loss: 2.3807
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 12.2 %, Loss: 2.4291
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 14.9 %, Loss: 2.0761
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 11.1 %, Loss: 2.6052
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 13.9 %, Loss: 2.0160
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 19.2 %, Loss: 1.8540
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 21.4 %, Loss: 1.9503
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 16.9 %, Loss: 2.1489
Epo

In [82]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [83]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 77.0 %


In [84]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [85]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.95_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 10.9 %, Loss: 2.1878
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.3 %, Loss: 2.0538
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 11.8 %, Loss: 2.2622
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 18.1 %, Loss: 2.1976
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 18.6 %, Loss: 2.1618
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 15.9 %, Loss: 1.9839
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 18.9 %, Loss: 2.3964
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 19.4 %, Loss: 2.1154
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 23.2 %, Loss: 2.3416
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 25.8 %, Loss: 1.7582
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 27.4 %, Loss: 1.7832
Ep

In [86]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [87]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 99.0 %


In [88]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [89]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.9_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 10.6 %, Loss: 2.3782
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.4 %, Loss: 2.0743
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 16.0 %, Loss: 1.7727
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 17.0 %, Loss: 2.0844
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.6 %, Loss: 2.2546
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 22.7 %, Loss: 1.7909
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 24.6 %, Loss: 1.9393
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 25.6 %, Loss: 2.4300
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 22.6 %, Loss: 1.6064
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 26.0 %, Loss: 1.5664
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 23.6 %, Loss: 2.2050
Ep

In [90]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [91]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 94.0 %


In [92]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [93]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.7_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.9 %, Loss: 2.8787
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.0 %, Loss: 2.2489
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 11.4 %, Loss: 2.3362
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 14.2 %, Loss: 1.8944
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 18.5 %, Loss: 1.9481
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 22.1 %, Loss: 2.1869
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 23.4 %, Loss: 2.3564
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 27.7 %, Loss: 1.8434
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 28.3 %, Loss: 2.0730
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 31.7 %, Loss: 1.6464
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 31.5 %, Loss: 1.9142
Epo

In [94]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [95]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 98.0 %


In [96]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [97]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_handsigns):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_HandSigns_100epochs_pr0.5_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 12.4 %, Loss: 2.1907
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 14.3 %, Loss: 2.0438
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 13.4 %, Loss: 2.1686
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 16.4 %, Loss: 2.1711
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.4 %, Loss: 1.8661
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 23.4 %, Loss: 2.4429
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 25.1 %, Loss: 2.1791
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 27.4 %, Loss: 2.4458
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 25.9 %, Loss: 2.7779
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 31.7 %, Loss: 1.6499
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 33.9 %, Loss: 1.6364
Ep

In [98]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [99]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_handsigns:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 100.0 %


In [100]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()